This series of exercises uses the mdp tool box developped at INRAE
    https://pymdptoolbox.readthedocs.io/en/latest/index.html
        
Install it (if not already installed) and import the toolbox and the examples


    
    

In [14]:
import mdptoolbox, mdptoolbox.example, mdptoolbox.util
import numpy as np



## Example: Forest Management

The considered problem is to manage a forest stand with first the objective to maintain an old forest for wildlife and second to make money selling cutwood. 

The forest stand is managed by two possible actions: Wait (action 0) or Cut (action 1). An action is decided and applied each time period  at the beginning of the period.

Three states are defined, corresponding to 3 age-class of trees: age-class 0-20years (state 1), 21-40 years (state 2), more than 40 years (state 3).  The state 3 correspond to the oldest age-class. 

At the end of a period t, if the state was s at t and action Wait is choosen, the state at the next time period will be min(s+ 1,3) if no fire occured (the forest grows). But there is a probability p that a fire burns the forest after the application of the action, leaving the stand at the youngest age-class (state 1). 

Let p = 0.1 be the probability of wildfire occurence during a time period. The problem is how to manage this stand in a long term vision to maximize the γ-discounted reward with γ= 0.95


mdptoolbox.example.forest() provides a modelisation of this problem in the MDP framework.


#### Q1. create and check  a mdp

In [3]:
# Creating a forest management problem with 3 states, a direct cut reward equal to 1 but for the last state (where it is equal to 2)
# a preservation reward equal to 4, and a probability of fire equal to 0.1
# (for more details,  look  at the "example" module)
    
P, R = mdptoolbox.example.forest(3,4,2,0.1)

# the following call checks that the MDP is correctly defined 
#(for more details,  look  at the "util" module : https://pymdptoolbox.readthedocs.io/en/latest/api/util.html)

mdptoolbox.util.check(P, R) # Nothing should happen


 Observe the transition matrix (P) and the reward matrix (R)   
 
 In particular check  probability to be in state 3 at time t+1 (s’ = 3), when being in state 3 (s = 3) and choosing action Wait (a = 0) at time t, is P(3,3,1) = 1 - p = 0.9 and the associated reward is R(3,1) = 4 

In [4]:
P



array([[[0.1, 0.9, 0. ],
        [0.1, 0. , 0.9],
        [0.1, 0. , 0.9]],

       [[1. , 0. , 0. ],
        [1. , 0. , 0. ],
        [1. , 0. , 0. ]]])

In [5]:
R

array([[0., 0.],
       [0., 1.],
       [4., 2.]])

#### Q2 Policy iteration 

Apply the policy iteration algorithm and look at  the policy computed

In [6]:
# Apply the policy iteration algorithm
P, R = mdptoolbox.example.forest(3,4,2,0.1)
pi = mdptoolbox.mdp.PolicyIteration(P, R, 0.9)
pi.setVerbose()
pi.run()


  Iteration		Number of different actions
    1		  1
    2		  0
Iterating stopped, unchanging policy found.


In [7]:
# display the policy associated to pi
pi.policy

(0, 0, 0)

In [8]:
# display  its value 

pi.V

(26.244000000000014, 29.484000000000016, 33.484000000000016)

Which is the minimal for probabilty of fire shall  that implies that it is better to cut   the forest  than wait in state 2 (age 21-40)





In [9]:
P, R = mdptoolbox.example.forest(3,4,2,0.1)
pi = mdptoolbox.mdp.PolicyIteration(P, R, 0.9)
pi.setVerbose()
pi.run()
pi.policy

  Iteration		Number of different actions
    1		  1
    2		  0
Iterating stopped, unchanging policy found.


(0, 0, 0)

#### Q3. Value iteration

Look at https://pymdptoolbox.readthedocs.io/en/latest/api/mdp.html#mdptoolbox.mdp.ValueIteration
    
And apply the Value iteration to the same problem (once with p=0.1, once with p=0.79) 

You can modify the  epsilon threshold and observe the number of iteration needed

In [10]:
P, R = mdptoolbox.example.forest(3,4,2,0.1)
pi = mdptoolbox.mdp.ValueIterationGS(P, R, 0.9, 0.01)
pi.setVerbose()
pi.run()
pi.policy
pi.V

  Iteration		V-variation
    1		  4.0
    2		  2.5029
    3		  0.9122714100000007
    4		  0.0489874447889993
    5		  0.01884470075378175
    6		  0.02190519408847269
    7		  0.019937784643719425
    8		  0.017800582030833567
    9		  0.01586472175726783
    10		  0.01413712536759526
    11		  0.012597471054032638
    12		  0.01122548314916827
    13		  0.010002916915990312
    14		  0.008913500145983022
    15		  0.007942731648864054
    16		  0.007077689460533776
    17		  0.006306858938941673
    18		  0.005619979500028904
    19		  0.005007908038937359
    20		  0.004462497225517836
    21		  0.003976487054679012
    22		  0.0035434082077507867
    23		  0.0031574959390354707
    24		  0.0028136133407379305
    25		  0.0025071829652532074
    26		  0.0022341258943576747
    27		  0.0019908074444501267
    28		  0.0017739887850112268
    29		  0.001580783826231169
    30		  0.0014086208021169
    31		  0.0012552080374490515
    32		  0.0011185034431555607
    33		  0.0009966873339

(25.5833879767579, 28.830654635546928, 32.83065463554693)

#### Q4.   Setting the parameters



You can also have a look at the cpu time neeed for the last run of the algorithm ; 

Considering a bigger problem (ex: 100 states),

P, R = mdptoolbox.example.forest(100,4,2,0.1)

which is the best policy (when to begin the cut ?)

which is the average time (on 10 runs) for each algo ?



In [13]:
P, R = mdptoolbox.example.forest(100,4,2,0.1)
#pi = mdptoolbox.mdp.ValueIterationGS(P, R, 0.9)
pi = mdptoolbox.mdp.PolicyIteration(P, R, 0.9)
pi.run()
#pi.policy
pi.time

0.24481964111328125

In [16]:
t = []
for i in range(1,100):
    P, R = mdptoolbox.example.forest(100,4,2,0.1)
    pi = mdptoolbox.mdp.PolicyIteration(P, R, 0.9)
    pi.run()
    t.append(pi.time)

t_mean = np.mean(t)
print(t_mean)

0.01185038113834882


When creating an instance of  ValueIterationGS  or of PolicyIteration, you can play with the parameters of the solving


class mdptoolbox.mdp.ValueIterationGS(transitions, reward, discount, epsilon=0.01, max_iter=10, initial_value=0, skip_check=False)

class mdptoolbox.mdp.PolicyIteration(transitions, reward, discount, policy0=None, max_iter=1000, eval_type=0, skip_check=False)

(look again at https://pymdptoolbox.readthedocs.io/en/latest/api/mdp.html)


Which is the best parametrization to have the quickest solving with   epsilon=0.01 ?